<a href="https://colab.research.google.com/github/eissa2002/Customer-Service-chatbot/blob/main/Customer_service_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# import necessary libraries
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import random
import json



In [32]:
# Load the intents file
with open('/content/intents.json') as file:
    data = json.load(file)

In [33]:
# Preprocess the data
stemmer = LancasterStemmer()

words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        # Tokenize the text and stem each word
        wrds = nltk.word_tokenize(pattern)
        wrds = [stemmer.stem(w) for w in wrds]
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent['tag'])

    # Add the label to the labels list
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [34]:
# Remove duplicates and sort the words
words = sorted(list(set(words)))

In [35]:
# Label encoding
training = []
output = []
out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    for w in words:
        if w in doc:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

training = np.array(training)
output = np.array(output)

In [36]:
# Building the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, input_shape=(len(training[0]),)),
    tf.keras.layers.Dense(8),
    tf.keras.layers.Dense(len(output[0]), activation='softmax'),
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [37]:
# Training the model
model.fit(training, output, epochs=300, batch_size=8)

Train on 106 samples
Epoch 1/300
106/106 [==============================] - 0s 1ms/sample - loss: 3.3190 - acc: 0.0472
Epoch 2/300
106/106 [==============================] - 0s 346us/sample - loss: 3.2904 - acc: 0.0755
Epoch 3/300
106/106 [==============================] - 0s 321us/sample - loss: 3.2639 - acc: 0.1132
Epoch 4/300
106/106 [==============================] - 0s 328us/sample - loss: 3.2385 - acc: 0.1132
Epoch 5/300
106/106 [==============================] - 0s 281us/sample - loss: 3.2107 - acc: 0.1509
Epoch 6/300
106/106 [==============================] - 0s 280us/sample - loss: 3.1834 - acc: 0.2075
Epoch 7/300
106/106 [==============================] - 0s 272us/sample - loss: 3.1533 - acc: 0.2170
Epoch 8/300
106/106 [==============================] - 0s 272us/sample - loss: 3.1230 - acc: 0.2170
Epoch 9/300
106/106 [==============================] - 0s 259us/sample - loss: 3.0883 - acc: 0.2264
Epoch 10/300
106/106 [==============================] - 0s 261us/sample - loss: 3

In [39]:
# Building the chatbot
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return np.array(bag)
def get_response(user_input):
    bag = bag_of_words(user_input, words)
    bag = np.reshape(bag, (1, len(bag)))
    results = model.predict(bag)
    results_index = np.argmax(results)
    tag = labels[results_index]

    for tg in data['intents']:
        if tg['tag'] == tag:
            responses = tg['responses']

    return random.choice(responses)



def chat():
    print("Start talking with the bot (type 'quit' to exit)")

    while True:
        user_input = input("You: ")
        if user_input.lower() == "quit":
            break

        response = get_response(user_input)
        print("zed:", response)
if __name__ == '__main__':
    chat()

Start talking with the bot (type 'quit' to exit)
You: hello
zed: Hi there
You: can u serve me
zed: Yes Sure, How can I support you
You: i have a shipment issue
zed: I'll double-check my work schedule
You: can i track my package
zed: Thank you for your order! Our standard shipping time is 3-5 business days. You can track your package by logging into your account on our website and clicking on the 'Track Order' button.
You: ok thank you
zed: Any time!
You: quit
